# Data processing with Python 

## Import modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os
import matplotlib as mpl
import pandas as pd        
import math

import sys
!{sys.executable} -m pip install pyFAI
import pyFAI
from pyFAI.gui import jupyter

!{sys.executable} -m pip install pybaselines
from pybaselines import Baseline
from pybaselines.utils import gaussian

pi = math.pi

# Load calibration file

calibrated using pyFAI-calib2 tool

pyFAI: https://pyfai.readthedocs.io/en/v2023.1/

In [ ]:
# load the calibration .poni file 
calibration = pyFAI.load('C:/poni/file/location/calibration.poni')
calibration

In [ ]:
#load mask for detector frame
oneD_mask = np.load("C:/mask/file/location/mask.npy")

## Specify folder of frames to read in

In [ ]:
# folder of nxs files to read in

collection_dir = "C:/collection/set/of/raw/nxs/files/location/"
count = 0
file_nxs = []

# Count the number of .nxs files in directory
for files in os.listdir(collection_dir):
    filename = os.fsdecode(files)
    if filename.endswith(".nxs"):
        file_nxs.append(filename)
        count += 1
print('File count:', count)
print(file_nxs)

In [ ]:
# making processing directory, change [0:-15] depending on you directory name to give it a reasonable name

processing_folder_dir = collection_dir[0:-15]
processing_folder = "overall_processed_patterns/"
processing_path = os.path.join(processing_folder_dir, processing_folder)

print("Processed pattern directory:", processing_path)

In [ ]:
if not os.path.exists(processing_path):
    os.makedirs(processing_path)

## Integration using pyFAI module
#### pyFAI: https://pyfai.readthedocs.io/en/v2023.1/

In [ ]:
# importing files and 1D integrating with pyFAI

frame_sized = np.zeros((count, 1441, 1440))
int_patterns = []
a = 0

while a < count:
    with h5py.File(collection_dir+file_nxs[a], 'r') as dat:
        frame = np.array(dat["/entry1/pixium_hdf/data"][()][:]) 
        frame_sized[a] = frame.reshape(frame.shape[1:])
        int_patterns.append(calibration.integrate1d(frame_sized[a], 1000, unit=pyFAI.units.TTH_DEG, radial_range=[4,30], mask=oneD_mask))
        a += 1

two_theta = int_patterns[0][0]

In [ ]:
# display all raw integrated patterns

large = []

# Create a new figure
plt.figure(figsize=(14, 14))
plt.xlabel('$2\\theta$ ($^{o}$)')
plt.xlim(4, 20)

plt.ylabel('Intensity')


for i in range(count):
    offset = i * 20  # Adjust the offset as needed
    plt.plot(two_theta, int_patterns[i][1] + offset, label=format(file_nxs[i]))
    large.append(max(int_patterns[i][1]+offset))

huge = max(large)
#plt.ylim(100,huge+20)
plt.legend()
plt.legend(loc='upper right')

In [ ]:
# use pattern name to remove any ones displaying no diffraction

remove = [61557 ,
          61558 ,
          61559 ,
          61560 ,
          61561 ,
          61562 ,
          61563]

# or remove none    
    
remove = []

In [ ]:
# remove files we don't want to process further

remove_set = set(remove)

filtered_file_list = [file_name for file_name in file_nxs if int(file_name.split('-')[-1].split('.')[0]) not in remove_set]

filtered_no = len(filtered_file_list)
print("Number of filtered files:", filtered_no)
print("Filtered file list:", filtered_file_list)

In [ ]:
# re integrate the chosen files 


filtered_frame_sized = np.zeros((count, 1441, 1440))
filtered_int_patterns = []
b = 0

for b in range(filtered_no):
    with h5py.File(collection_dir+filtered_file_list[b], 'r') as dat:
        frame = np.array(dat["/entry1/pixium_hdf/data"][()][:]) 
        filtered_frame_sized[b] = frame.reshape(frame.shape[1:])
        filtered_int_patterns.append(calibration.integrate1d(filtered_frame_sized[b], 1000, unit=pyFAI.units.TTH_DEG, radial_range=[4,30], mask=oneD_mask))
       # filtered_int_patterns.append(calibration.integrate1d(filtered_frame_sized[b], 1000, unit=pyFAI.units.TTH_DEG, radial_range=[4,30]))

two_theta2 = filtered_int_patterns[0][0]

In [ ]:
# display the final files to merge and process

large2 = []

# Create a new figure
plt.figure(figsize=(14, 14))
plt.xlabel('$2\\theta$ ($^{o}$)')
plt.xlim(4, 20)

plt.ylabel('Intensity')


for j in range(filtered_no):
    offset = j * 20  # Adjust the offset as needed
    plt.plot(two_theta2, filtered_int_patterns[j][1] + offset, label=format(filtered_file_list[j]))
    large2.append(max(filtered_int_patterns[j][1]+offset))

huge2 = max(large2)
plt.ylim(100,huge2+20)
plt.legend()
plt.legend(loc='upper right')

In [ ]:
# summing and averaging the raw patterns

raw_intensities = []

for k in range(filtered_no):
    raw_intensities.append(filtered_int_patterns[k][1])
    
added = np.sum(raw_intensities, axis=0)
averaged_intensities = added/filtered_no

In [ ]:
# display the raw merged pattern

plt.figure(figsize=(14, 14))
plt.xlabel('$2\\theta$ ($^{o}$)')
plt.xlim(4, 20)
plt.ylim(80, 250)
plt.ylabel('Intensity')

plt.plot(two_theta2, averaged_intensities)


# Baseline correction 

### using https://pybaselines.readthedocs.io/en/latest/ mor() baseline correction

In [ ]:
# baseline correction set-up

x = two_theta2
y = averaged_intensities

baseline_fitter = Baseline(x_data=x)

In [ ]:
# display baseline correction and edit half_window for a best fit

half_window = 4

plt.figure()
plt.figure(figsize=(14, 14))
plt.plot(x, y, label='data')
plt.plot(x, baseline_fitter.mor(y, half_window=half_window)[0], label=f'half_window={half_window}')
plt.xlabel('$2\\theta$ ($^{o}$)')
plt.xlim(4, 20)
#plt.ylim(80, 200)
plt.ylabel('Intensity')
plt.legend()

In [ ]:
# baseline correction

corrected_data = y - baseline_fitter.mor(y, half_window=half_window)[0]

In [ ]:
# display all baseline correction stages

plt.figure()
plt.figure(figsize=(14, 14))
plt.plot(x, y, label='data')
plt.plot(x, baseline_fitter.mor(y, half_window=half_window)[0], label=f'half_window={half_window}')
plt.plot(x, (corrected_data*20)+75, label='corrected_baseline')
plt.xlabel('$2\\theta$ ($^{o}$)')
plt.ylabel('Intensity /a.u.')
plt.xlim(5, 20)
plt.ylim(70, 200)
plt.legend()
#plt.savefig(processing_path + twoD_pattern_name_baseline_corr)

In [ ]:
data_name = modified_datasets_name + "_fast_raw_merged_baselineCorr.xy"
print(data_name)

In [ ]:
# export final baseline corrected merged pattern for the collection set
np.savetxt(processing_path+data_name, np.c_[x, corrected_data])